In [1]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
path_tounzipfile = 'C:/Users/Subhadri/Desktop/Data Science/Term 1/Data Scrapping & Data Wranglig/geckodriver-v0.25.0-win64'
os.chdir(path_tounzipfile)
browser = webdriver.Firefox(path_tounzipfile)

In [3]:
def get_data(url):
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    ## Extracting the data from all the pages
    div = soup.find_all('div', class_ = 'col col-7-12')
    mob_name = []
    color = []
    no_of_ratings =[]
    ratings = []
    
    n = []
    for i in div:
        n.append(i.find_all('div', class_ = '_3wU53n'))
    a = [k for j in n for k in j]
    data_split = []
    for i in a:
        data_split.append(i.text.strip().split('('))
    for i in data_split:
        mob_name.append(i[0].rstrip())
    #mob_name
    
    x = []
    for i in data_split:
        x.append(i[1].split(','))
    for j in x:
        color.append(j[0])
    #color
    
    d = []
    for i in div:
        d.append(i.find_all('li', class_ = 'tVe95H'))
    desc = []
    for j in d:
        desc.append(j[0].text.strip())
    ram = []
    storage = []
    for k in desc:
        ram.append(k.split('|')[0][:-5])
        storage.append(k.split('|')[1][:-5])
    #ram
    #storage
    
    y = []
    for i in div:
        y.append(i.find_all('span', class_ = '_38sUEc'))
    v = []
    for j in div:
        try:
            v.append(j.find('span',class_='_38sUEc').find_next('span').text.strip().split()[0])
        except:
            v.append('NA')
    for l in v:
        no_of_ratings.append(l.replace('NA','0').replace(',',''))
    #no_of_ratings
    
    z = []
    for i in div:
        z.append(i.find_all('div', class_ = 'hGSR34'))
    r = []
    for j in div:
        try:
            r.append(j.find('div', class_ = 'hGSR34').text.strip().split()[0])
        except:
            r.append('NA')
    for k in r:
        ratings.append(k.replace('NA','0'))
    #ratings
    
    data_frame = pd.DataFrame({'Phone':mob_name, 'Color':color, 'RAM':ram, 'Storage':storage,
                               'No. of ratings':no_of_ratings, 'Rating':ratings})
    return data_frame

url = 'https://www.flipkart.com/mobiles/mi~brand/pr?sid=tyy,4io&otracker=nmenu_sub_Electronics_0_Mi&page={}'    
phones = {}
for i in range(1,12):
    pages = url.format(i)
    phones[i] = get_data(pages)

In [4]:
mi_phones = pd.concat([phones[i] for i in phones], ignore_index=True)
mi_phones

,Phone,Color,RAM,Storage,No. of ratings,Rating
0,Redmi 8,Onyx Black,4 GB,64 GB,489549,4.4
1,Redmi 8,Sapphire Blue,4 GB,64 GB,489549,4.4
2,Redmi Note 7 Pro,Space Black,4 GB,64 GB,790116,4.5
3,Redmi 8A,Midnight Black,2 GB,32 GB,127061,4.4
4,Redmi 8A,Ocean Blue,3 GB,32 GB,113419,4.4
...,...,...,...,...,...,...
142,Redmi 5A,Gold,2 GB,16 GB,1216650,4.5
143,Redmi 3S,Gold,2 GB,16 GB,211541,4.3
144,Redmi Y2,Black,3 GB,32 GB,115053,4.5
145,Redmi Note 5,Blue,3 GB,32 GB,348582,4.4


In [5]:
mi_phones['No. of ratings'] = mi_phones['No. of ratings'].astype('int')
mi_phones['Rating'] = mi_phones['Rating'].astype('float')
mi_phones.head()

,Phone,Color,RAM,Storage,No. of ratings,Rating
0,Redmi 8,Onyx Black,4 GB,64 GB,489549,4.4
1,Redmi 8,Sapphire Blue,4 GB,64 GB,489549,4.4
2,Redmi Note 7 Pro,Space Black,4 GB,64 GB,790116,4.5
3,Redmi 8A,Midnight Black,2 GB,32 GB,127061,4.4
4,Redmi 8A,Ocean Blue,3 GB,32 GB,113419,4.4


In [6]:
## No. of unique 'mi' phones

mi_phones['Phone'].nunique()

38

In [7]:
mi_phones['Phone'].unique()

array(['Redmi 8', 'Redmi Note 7 Pro', 'Redmi 8A', 'Redmi Note 7S',
       'Redmi 7A', 'Redmi Go', 'Redmi Note 8', 'Mi A2',
       'Redmi Note 9 Pro', 'Redmi 6 Pro', 'Redmi 4A', 'Redmi Note 7',
       'Redmi K20', 'Redmi Note 5', 'Redmi 6A', 'Redmi 7', 'Redmi Y1',
       'Redmi 6', 'Redmi Y1 lite', 'Redmi 3S Prime', 'Redmi 5', 'MI3',
       'Mi 4i', 'Redmi Note 6 Pro', 'Redmi Note 4', 'Redmi Note 5 Pro',
       'Redmi 4', 'Redmi 2 Prime', 'Redmi Y2', 'Redmi 2', 'Mi Max',
       'Mi 5', 'Mi Max 2', 'MI4', 'Redmi Y3', 'Redmi 1S', 'Redmi 5A',
       'Redmi 3S'], dtype=object)

In [8]:
var1 = mi_phones.groupby(['Phone', 'RAM'])['Rating'].mean().sort_values(ascending = False).reset_index()
var1[var1['Rating'] == var1['Rating']].head()

,Phone,RAM,Rating
0,Redmi Note 9 Pro,6 GB,4.9
1,Redmi Note 7 Pro,4 GB,4.5
2,Redmi Y2,3 GB,4.5
3,Redmi K20,6 GB,4.5
4,Redmi Note 5 Pro,4 GB,4.5


### I won't be considering Redmi Note 9 Pro as it has very less number of ratings and 0 reviews.

In [9]:
var1[var1['Rating'] < var1['Rating'].max()]

,Phone,RAM,Rating
1,Redmi Note 7 Pro,4 GB,4.5
2,Redmi Y2,3 GB,4.5
3,Redmi K20,6 GB,4.5
4,Redmi Note 5 Pro,4 GB,4.5
5,Redmi 5A,2 GB,4.5
6,Redmi Note 9 Pro,4 GB,4.5
7,Redmi Note 5 Pro,6 GB,4.5
8,Redmi Note 6 Pro,4 GB,4.5
9,Redmi Note 7,4 GB,4.5
10,Redmi Note 7 Pro,6 GB,4.5


In [10]:
var1[var1['Rating'] == 4.5] ## Taking the second highest rating

,Phone,RAM,Rating
1,Redmi Note 7 Pro,4 GB,4.5
2,Redmi Y2,3 GB,4.5
3,Redmi K20,6 GB,4.5
4,Redmi Note 5 Pro,4 GB,4.5
5,Redmi 5A,2 GB,4.5
6,Redmi Note 9 Pro,4 GB,4.5
7,Redmi Note 5 Pro,6 GB,4.5
8,Redmi Note 6 Pro,4 GB,4.5
9,Redmi Note 7,4 GB,4.5
10,Redmi Note 7 Pro,6 GB,4.5


### These phones have the highest rating. In order to choose the best amongst these phones, we need to check which phone has the highest number of ratings.

In [11]:
var2 = mi_phones[mi_phones['Phone'].isin(['Redmi K20 Pro','Redmi Note 8 Pro', 'Redmi Note 7 Pro', 'Redmi Note 6 Pro', 
                                          'Redmi Note 5 Pro', 'Redmi K20', 'Redmi Y2', 'Redmi 5A', 'Redmi Note 7', 
                                          'Redmi Note 9 Pro'])].groupby(['Phone', 'RAM'])['No. of ratings'].max().\
sort_values(ascending = False).reset_index()
var2

,Phone,RAM,No. of ratings
0,Redmi 5A,2 GB,1216650
1,Redmi Note 5 Pro,4 GB,1153581
2,Redmi Note 7 Pro,4 GB,790116
3,Redmi Note 6 Pro,4 GB,609423
4,Redmi 5A,3 GB,596984
5,Redmi Note 7 Pro,6 GB,259434
6,Redmi Note 5 Pro,6 GB,252714
7,Redmi Note 7,4 GB,173842
8,Redmi Note 7,3 GB,129532
9,Redmi Y2,3 GB,115053


### Since, Redmi 5A with RAM 3 GB is not in the dataframe 'var1', so we don't need it to be there in 'var2' dataframe as all other phones in the 'var2' dataframe are exactly the same as 'var1' dtaframe.

In [12]:
var2 = var2.drop(4)
var2

,Phone,RAM,No. of ratings
0,Redmi 5A,2 GB,1216650
1,Redmi Note 5 Pro,4 GB,1153581
2,Redmi Note 7 Pro,4 GB,790116
3,Redmi Note 6 Pro,4 GB,609423
5,Redmi Note 7 Pro,6 GB,259434
6,Redmi Note 5 Pro,6 GB,252714
7,Redmi Note 7,4 GB,173842
8,Redmi Note 7,3 GB,129532
9,Redmi Y2,3 GB,115053
10,Redmi K20,6 GB,44474


In [13]:
var2.groupby('Phone')['No. of ratings'].sum().sort_values(ascending = False).reset_index()

,Phone,No. of ratings
0,Redmi Note 5 Pro,1406295
1,Redmi 5A,1216650
2,Redmi Note 7 Pro,1049550
3,Redmi Note 6 Pro,609423
4,Redmi Note 7,303374
5,Redmi Y2,140518
6,Redmi K20,44474
7,Redmi Note 9 Pro,20


### The summation of the number of ratings of Redmi Note 5 Pro of all the variants is the highest. So, I need to scrape the reviews of Redmi Note 5 Pro for the variants 4 GB RAM | 64 GB ROM and  6 GB RAM | 64 GB ROM separately and then merge their reviews into a single dataframe to move ahead.